# 🧹 Práctica Integrada: Pipeline de Preprocesamiento para Análisis de Feedback en E-Commerce


### 🛍️ Contexto Empresarial
MarketMind analiza comentarios de clientes y tweets en inglés y español. El modelo fine-tuned de GPT-4 genera resúmenes erróneos debido a entradas mal preprocesadas: URLs, precios y emoticones sin normalizar.

Esta práctica busca crear un pipeline robusto que preserve formatos útiles y limpie lo irrelevante.


## 🔧 Fase 1: Limpieza Contextual de Texto

In [ ]:

import json
import re
import emoji

# Cargar datos
with open("validation_samples.json", "r", encoding="utf-8") as f:
    samples = json.load(f)
print(samples[0])


In [ ]:

def limpiar_texto(texto):
    # Eliminar URLs y handles
    texto = re.sub(r"https?://\S+|www\.\S+", "", texto)
    texto = re.sub(r"@\w+", "", texto)
    
    # Eliminar signos de puntuación no deseados (excepto ! ? % $ /)
    texto = re.sub(r"[\.,;:()\[\]"'\\]", "", texto)
    
    # Preservar emoticones con emoji
    texto = emoji.demojize(texto, delimiters=("", ""))  # No los elimina
    
    # Unificar espacios
    texto = re.sub(r"\s+", " ", texto).strip()
    
    return texto


In [ ]:

texto_prueba = "🔥 OFERTA Compre 2x zapatos Nike a $99.99 antes $150 👟\nVálido hasta el 30/11/2023 Atención ¿Envío gratis 😃 #ModaDeportiva2023"
print(limpiar_texto(texto_prueba))


## 🔄 Fase 2: Normalización de Números y Unidades

In [ ]:

import dateparser

def normalizar_numeros(texto):
    # Normalizar fechas
    fechas = re.findall(r"\b\d{2}/\d{2}/\d{4}\b", texto)
    for fecha in fechas:
        fecha_iso = dateparser.parse(fecha).strftime("%Y-%m-%d")
        texto = texto.replace(fecha, fecha_iso)

    # Monedas
    texto = re.sub(r"\$\s?(\d+(\.\d{1,2})?)", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(\.\d{1,2})?)\$", r"<USD>\1", texto)
    texto = re.sub(r"(\d+(\.\d{1,2})?)€", r"<EUR>\1", texto)
    
    # Unidades
    texto = re.sub(r"\b(\d+)(x|kg|ml)\b", r"\1_\2", texto)
    
    # Reemplazar números genéricos que no sean parte de lo anterior
    texto = re.sub(r"(?<!<USD>|<EUR>|\d_)(?<!\d/)\b\d{2,}\b(?!%)", "<NUM>", texto)
    
    return texto


In [ ]:

texto2 = limpiar_texto(texto_prueba)
print(normalizar_numeros(texto2))


## 🔤 Fase 3: Normalización de Mayúsculas con Reconocimiento de Entidades

In [ ]:

import spacy
nlp = spacy.load("es_core_news_lg")

# Diccionario personalizado de marcas (simulado)
marcas_manual = {"Zara", "Nike", "Adidas", "iPhone"}

def normalizar_mayusculas(texto):
    doc = nlp(texto)
    resultado = []

    for token in doc:
        if token.text in marcas_manual:
            resultado.append(token.text)
        elif token.ent_type_ in {"ORG", "LOC", "PRODUCT", "GPE"}:
            resultado.append(token.text)
        elif token.text.startswith("#"):
            resultado.append(token.text)
        else:
            resultado.append(token.text.lower())
    
    return " ".join(resultado)


In [ ]:

texto_final = normalizar_mayusculas(normalizar_numeros(limpiar_texto(texto_prueba)))
print(texto_final)


## ✅ Verificación Final


El resultado esperado debe conservar formatos y significado semántico útil para análisis de feedback comercial.  
Ejemplo de salida:

```
🔥 oferta compre 2_unidades zapatos Nike a <USD>99.99 antes <NUM> 👟
válido hasta el 2023-11-30 atención ¿envío gratis 😃 #ModaDeportiva2023
```
